# Setup

In [1]:
import os
import tweepy
import sys
import csv
import gzip
import pandas as pd
from tweepy import TweepError

# User defined constants

In [8]:
# Specify the name of the CSV file containing the list of usernames
USERNAME_FILE = "./Social Media Mining_April 1, 2019_09.08.csv"
OUTPUT_USERNAME_FILE = 'valid_usernames.csv'
OUTPUT_UID_FILE = 'valid_uids'
OUTPUT_UID_ZIP_FILE = 'valid_uids.gz'

# Helper functions

In [3]:
def auth_tweepy(token_label_string, token_object):
    auth_handler = tweepy.OAuthHandler(token_object['consumer-key'], token_object['consumer-secret'])
    auth_handler.set_access_token(token_object['access-token'], token_object['access-token-secret'])
    
    twitter_api = tweepy.API(auth_handler)

    # twitter_api_tokens.insert(0, (token_label_string, token_object))
    
    return twitter_api

In [4]:
def get_user_id(username, api):
    user = api.get_user(username)
    return user.id

# Authenticate with twitter

In [5]:
token_label_string, token_object = 'non-bs detector 2', {
        'consumer-key': "NqBkDNnFE843DaKtsFsPJCs2U",
        'consumer-secret': "UbyFnnsJfc3rLkgf7qcoel9QZ8yJtv8FCs1JLySjC9eUIcgMdi",
        'access-token': "1559190498-aSi2gUaawU9KioPU02vyg3ZCBRftxE6oqepyxu7",
        'access-token-secret': "ZDfrZOJ4Sq4ZWqcE3gftpgXqAfh2PWyhLw3rRFUf6nEoH"
    }
twitter_api = auth_tweepy(token_label_string, token_object)

# Process usernames

In [6]:
uids = []
valid_names = []
for username in pd.read_csv(USERNAME_FILE).TwitHandle:
    username = str(username)
    pos = username.find('@')
    if not pos == -1:
        username = username[pos+1:]
    if username == 'nan':
        continue
    if username.lower() == 'yes':
        continue
    try:
        uid = get_user_id(username, twitter_api)
    except TweepError:
        # Not useful
        continue
    uids.append(str(uid))
    valid_names.append(username)

# Output valid usernames

In [19]:
with open(OUTPUT_USERNAME_FILE, 'w') as csvfile:
    userwriter = csv.writer(csvfile)
    userwriter.writerow(valid_names)

with open(OUTPUT_UID_FILE, "w") as f:
    for uid in uids:
        f.write('"' + str(uid) + '"\n')

f_in = open(OUTPUT_UID_FILE, 'rb')
f_out = gzip.open(OUTPUT_UID_ZIP_FILE, 'wb')
f_out.write(f_in.read())
f_out.close()
f_in.close()